# 曆象推算
從[中央氣象局](http://www.cwb.gov.tw/V7/astronomy/calendar.htm#s1)我們可以查得曆象資料，包括24節氣、陰陽曆對照表、朔望月、日蝕月蝕。

我們知道，
* 太陽的黃經在15度的整數倍數的時候就是24節氣的日子，
* 太陽與月亮同一黃經的時候就是朔月的日子，太陽與月亮黃經相差180度的時候就是朔月的日子，
* 太陽、地球、月亮幾乎共線的時候會產生日蝕或月蝕。

所以，
* 只要知道太陽月亮在黃道座標系統中每一時刻的位置，就可以推算出24節氣、朔望月的時刻。
* 再加上知道陰曆置潤規則就可以得到陰陽曆對照表。
* 只要知道太陽月亮在任何座標系統中每一時刻的位置，就可以推算出日蝕月蝕。

後面，我們要一一的加以推算。

In [1]:
from astropy.time import Time,TimeDelta
from astropy.coordinates import get_sun,FK5,PrecessedGeocentric,get_body
from astropy import coordinates as coord
from astropy import constants as const
from astropy.coordinates import SkyCoord
import math
import numpy as np


In [2]:
jieqinames=["春分","清明","穀雨","立夏","小滿","芒種","夏至","小暑","大暑","立秋",
              "處暑","白露","秋分","寒露","霜降","立冬","小雪","大雪","冬至","小寒","大寒","立春","雨水","驚蟄"]
moonshapenames=['朔月','上弦','望月','下弦']
oneday=TimeDelta(1, format='jd')
eighthours=TimeDelta(8*3600,format='sec')
onehour=TimeDelta(3600, format='sec')
oneminute=TimeDelta(60, format='sec')
onesecond=TimeDelta(1, format='sec')
def diff(d1,d2,mod):
  dif=(d1-d2) % mod
  if dif>mod/2.0:
    dif=dif-mod
  return dif
# CCTerm 用來計算從起始日(start)到結束日(end)這段時間的節氣(type='jieqi')或朔望月(type='moon')的日子。
# 函數值編號(code)與日期(date)所組成。
def CCTerm(start,end,type,detail=False):
    def nearestday(t):
        t2=Time(Time(t,out_subfmt='date').iso)
        if (t-t2).value>0.5:
            t=t+TimeDelta(0.5,format='jd')
            t2=Time(Time(t,out_subfmt='date').iso)
        return t2

    def termtime(daytime,unit='hour'):
        if unit=='hour':
            times=daytime+onehour*np.linspace(0,24,25)
        elif unit=='minute':
            times=daytime+oneminute*np.linspace(0,60,61)
        else:
            times=daytime+onesecond*np.linspace(0,60,61)
        gmttimes=times-eighthours
        if type=='jieqi':
            diflongitudes=get_body('sun', gmttimes,  ephemeris='jpl').\
                transform_to(coord.GeocentricTrueEcliptic(equinox=gmttimes)).lon.deg
        else:
            sunlongitudes=get_body('sun', gmttimes,  ephemeris='jpl').\
                transform_to(coord.GeocentricTrueEcliptic(equinox=gmttimes)).lon.deg
            moonlongitudes=get_body('moon', gmttimes,  ephemeris='jpl').\
                transform_to(coord.GeocentricTrueEcliptic(equinox=gmttimes)).lon.deg
            diflongitudes=(moonlongitudes-sunlongitudes)%360
        difmod=diflongitudes % sec
        for i in range(0,len(diflongitudes)-1):
            d1=diff(difmod[i],0,sec)
            d2=diff(difmod[i+1],0,sec)
            if d1<=0 and d2>0:
                if unit=='hour':
                    return(termtime(times[i],unit='minute')) 
                elif unit=='minute':
                    return(termtime(times[i],unit='second')) 
                else:
                    return((int(math.floor(diflongitudes[i+1]/sec)),times[i].value))

        return((-1,''))
    #print(type)
    daycount=int(round((Time(end)-Time(start)).value))
    #print(daycount)
    times=Time(start)+oneday*np.linspace(0,daycount,daycount+1)
    times=Time(Time(times,out_subfmt='date').iso)
    gmttimes=times-eighthours
    if type=='jieqi':
        diflongitudes= get_body('sun', gmttimes,  ephemeris='jpl').\
            transform_to(coord.GeocentricTrueEcliptic(equinox=gmttimes)).lon.deg
        sec=15
    else:
        sunlongitudes=get_body('sun', gmttimes,  ephemeris='jpl').\
            transform_to(coord.GeocentricTrueEcliptic(equinox=gmttimes)).lon.deg
        moonlongitudes=get_body('moon', gmttimes,  ephemeris='jpl').\
            transform_to(coord.GeocentricTrueEcliptic(equinox=gmttimes)).lon.deg
        diflongitudes=(moonlongitudes-sunlongitudes)%360
        sec=90
    res=[]
    difmod=diflongitudes % sec
    for i in range(0,len(diflongitudes)-1):
        d1=diff(difmod[i],0,sec)
        d2=diff(difmod[i+1],0,sec)
        if d1<=0 and d2>0:
            if detail==True:
                res.append(termtime(times[i],'hour'))
            else:
                times.out_subfmt='date'
                res.append((int(math.floor(diflongitudes[i+1]/sec)),times[i].value))
    return(res)



In [3]:
jieqi=CCTerm('2016-1-1','2017-3-1','jieqi')
darkmoon=CCTerm('2016-1-1','2017-3-1','moon')
for code,date in jieqi:
    print(jieqinames[code]+' '+date)
for code,date in darkmoon:
    print(moonshapenames[code]+' '+date)
jieqi=CCTerm('2016-1-1','2017-3-1','jieqi',True)
darkmoon=CCTerm('2016-1-1','2017-3-1','moon',True)
for code,date in jieqi:
    print(jieqinames[code]+' '+date)
for code,date in darkmoon:
    print(moonshapenames[code]+' '+date)
jieqi=CCTerm('2033-1-1','2034-3-31','jieqi',True)
darkmoon=CCTerm('2033-1-1','2034-3-31','moon',True)
for code,date in jieqi:
    print(jieqinames[code]+' '+date)
for code,date in darkmoon:
    print(moonshapenames[code]+' '+date)
        

小寒 2016-01-06
大寒 2016-01-20
立春 2016-02-04
雨水 2016-02-19
驚蟄 2016-03-05
春分 2016-03-20
清明 2016-04-04
穀雨 2016-04-19
立夏 2016-05-05
小滿 2016-05-20
芒種 2016-06-05
夏至 2016-06-21
小暑 2016-07-07
大暑 2016-07-22
立秋 2016-08-07
處暑 2016-08-23
白露 2016-09-07
秋分 2016-09-22
寒露 2016-10-08
霜降 2016-10-23
立冬 2016-11-07
小雪 2016-11-22
大雪 2016-12-07
冬至 2016-12-21
小寒 2017-01-05
大寒 2017-01-20
立春 2017-02-03
雨水 2017-02-18
下弦 2016-01-02
朔月 2016-01-10
上弦 2016-01-17
望月 2016-01-24
下弦 2016-02-01
朔月 2016-02-08
上弦 2016-02-15
望月 2016-02-23
下弦 2016-03-02
朔月 2016-03-09
上弦 2016-03-16
望月 2016-03-23
下弦 2016-03-31
朔月 2016-04-07
上弦 2016-04-14
望月 2016-04-22
下弦 2016-04-30
朔月 2016-05-07
上弦 2016-05-14
望月 2016-05-22
下弦 2016-05-29
朔月 2016-06-05
上弦 2016-06-12
望月 2016-06-20
下弦 2016-06-28
朔月 2016-07-04
上弦 2016-07-12
望月 2016-07-20
下弦 2016-07-27
朔月 2016-08-03
上弦 2016-08-11
望月 2016-08-18
下弦 2016-08-25
朔月 2016-09-01
上弦 2016-09-09
望月 2016-09-17
下弦 2016-09-23
朔月 2016-10-01
上弦 2016-10-09
望月 2016-10-16
下弦 2016-10-23
朔月 2016-10-31
上弦 2016-11-08
望月 201

小寒 2033-01-05 09:08:06.000
大寒 2033-01-20 02:32:47.000
立春 2033-02-03 20:41:35.000
雨水 2033-02-18 16:33:48.000
驚蟄 2033-03-05 14:32:20.000
春分 2033-03-20 15:22:43.000
清明 2033-04-04 19:08:07.000
穀雨 2033-04-20 02:13:07.000
立夏 2033-05-05 12:13:46.000
小滿 2033-05-21 01:10:59.000
芒種 2033-06-05 16:13:26.000
夏至 2033-06-21 09:01:08.000
小暑 2033-07-07 02:24:58.000
大暑 2033-07-22 19:52:49.000
立秋 2033-08-07 12:15:45.000
處暑 2033-08-23 03:01:51.000
白露 2033-09-07 15:20:22.000
秋分 2033-09-23 00:51:41.000
寒露 2033-10-08 07:13:57.000
霜降 2033-10-23 10:27:37.000
立冬 2033-11-07 10:41:05.000
小雪 2033-11-22 08:16:10.000
大雪 2033-12-07 03:44:56.000
冬至 2033-12-21 21:46:00.000
小寒 2034-01-05 15:04:31.000
大寒 2034-01-20 08:27:17.000
立春 2034-02-04 02:41:10.000
雨水 2034-02-18 22:30:11.000
驚蟄 2034-03-05 20:32:23.000
春分 2034-03-20 21:17:30.000
朔月 2033-01-01 18:16:55.000
上弦 2033-01-08 11:34:26.000
望月 2033-01-15 21:07:14.000
下弦 2033-01-24 01:45:45.000
朔月 2033-01-31 05:59:48.000
上弦 2033-02-06 21:34:16.000
望月 2033-02-14 15:04:18.000
下

In [4]:
darkmoon=CCTerm('2016-1-1','2017-3-1','moon',True)
for code,date in darkmoon:
    print(moonshapenames[code]+' '+date)
darkmoon=CCTerm('2016-1-1','2017-3-1','moon')
for code,date in darkmoon:
    print(moonshapenames[code]+' '+date)

下弦 2016-01-02 13:30:23.000
朔月 2016-01-10 09:30:30.000
上弦 2016-01-17 07:26:24.000
望月 2016-01-24 09:45:44.000
下弦 2016-02-01 11:27:47.000
朔月 2016-02-08 22:38:54.000
上弦 2016-02-15 15:46:31.000
望月 2016-02-23 02:19:51.000
下弦 2016-03-02 07:10:39.000
朔月 2016-03-09 09:54:28.000
上弦 2016-03-16 01:02:56.000
望月 2016-03-23 20:00:50.000
下弦 2016-03-31 23:16:48.000
朔月 2016-04-07 19:23:40.000
上弦 2016-04-14 11:59:20.000
望月 2016-04-22 13:23:37.000
下弦 2016-04-30 11:28:42.000
朔月 2016-05-07 03:29:30.000
上弦 2016-05-14 01:02:07.000
望月 2016-05-22 05:14:26.000
下弦 2016-05-29 20:11:59.000
朔月 2016-06-05 10:59:35.000
上弦 2016-06-12 16:09:47.000
望月 2016-06-20 19:02:20.000
下弦 2016-06-28 02:18:42.000
朔月 2016-07-04 19:00:57.000
上弦 2016-07-12 08:51:46.000
望月 2016-07-20 06:56:38.000
下弦 2016-07-27 06:59:43.000
朔月 2016-08-03 04:44:28.000
上弦 2016-08-11 02:20:48.000
望月 2016-08-18 17:26:35.000
下弦 2016-08-25 11:40:54.000
朔月 2016-09-01 17:03:05.000
上弦 2016-09-09 19:48:47.000
望月 2016-09-17 03:05:07.000
下弦 2016-09-23 17:56:08.000
朔

In [5]:
Cnumber=['一','二','三','四','五','六','七',
               '八','九','十','十一','十二']
def _CCalendar(year):
    limits=CCTerm(str(year-1)+'-12-1',str(year+1)+'-1-1','jieqi')
    limits=[data for data in limits if data[0]==18]
    darkmoon=CCTerm(Time(limits[0][1])+oneday,Time(limits[1][1])+oneday,'moon')
    #print(darkmoon)
    darkmoon=[m[1] for m in darkmoon if m[0]==0]
    #print(darkmoon)
    res=[]
    #print(len(darkmoon))
    if len(darkmoon)<13:
        for i in range(12):
            j=(i+12)%12
            if j==0:
                j=12
            res.append((j,darkmoon[i]))
    else:
        add=False
        mnumber=12
        for i in range(12):
            js=CCTerm(Time(darkmoon[i]),Time(darkmoon[i+1]),'jieqi')
            #print(js)
            js=[d for d in js if d[0]%2==0]
            #print(js)
            if add==False and len(js)==0 :
                res.append((mnumber-1+0.1,darkmoon[i]))
                add=True
            else:
                res.append((mnumber,darkmoon[i]))
                mnumber=mnumber+1
                if mnumber==13:
                    mnumber=1
        res.append((mnumber,darkmoon[12]))
    return(res)
def CCalendar(year):
    res=_CCalendar(year)+_CCalendar(year+1)
    return([m for m in res if m[1]>=str(year)+'-01-01' and m[1]<str(year+1)+'-01-01'])

def PrintCCalender(year):
    res=CCalendar(year)
    for m in res:
        name=Cnumber[int(m[0])-1]+'月初一'
        if int(m[0])!=m[0]:
            name='潤'+name
        print(name+','+m[1])
    



In [6]:
PrintCCalender(2016)
PrintCCalender(2033)
PrintCCalender(2034)


十二月初一,2016-01-10
一月初一,2016-02-08
二月初一,2016-03-09
三月初一,2016-04-07
四月初一,2016-05-07
五月初一,2016-06-05
六月初一,2016-07-04
七月初一,2016-08-03
八月初一,2016-09-01
九月初一,2016-10-01
十月初一,2016-10-31
十一月初一,2016-11-29
十二月初一,2016-12-29


十二月初一,2033-01-01
一月初一,2033-01-31
二月初一,2033-03-01
三月初一,2033-03-31
四月初一,2033-04-29
五月初一,2033-05-28
六月初一,2033-06-27
七月初一,2033-07-26
八月初一,2033-08-25
九月初一,2033-09-23
十月初一,2033-10-23
十一月初一,2033-11-22
潤十一月初一,2033-12-22
十二月初一,2034-01-20
一月初一,2034-02-19
二月初一,2034-03-20
三月初一,2034-04-19
四月初一,2034-05-18
五月初一,2034-06-16
六月初一,2034-07-16
七月初一,2034-08-14
八月初一,2034-09-13
九月初一,2034-10-12
十月初一,2034-11-11
十一月初一,2034-12-11


In [7]:
PrintCCalender(2017)

一月初一,2017-01-28
二月初一,2017-02-26
三月初一,2017-03-28
四月初一,2017-04-26
五月初一,2017-05-26
六月初一,2017-06-24
潤六月初一,2017-07-23
七月初一,2017-08-22
八月初一,2017-09-20
九月初一,2017-10-20
十月初一,2017-11-18
十一月初一,2017-12-18


In [8]:
PrintCCalender(2018)

十二月初一,2018-01-17
一月初一,2018-02-16
二月初一,2018-03-17
三月初一,2018-04-16
四月初一,2018-05-15
五月初一,2018-06-14
六月初一,2018-07-13
七月初一,2018-08-11
八月初一,2018-09-10
九月初一,2018-10-09
十月初一,2018-11-08
十一月初一,2018-12-07


In [9]:
PrintCCalender(2013)

十二月初一,2013-01-12
一月初一,2013-02-10
二月初一,2013-03-12
三月初一,2013-04-10
四月初一,2013-05-10
五月初一,2013-06-08
六月初一,2013-07-08
七月初一,2013-08-07
八月初一,2013-09-05
九月初一,2013-10-05
十月初一,2013-11-03
十一月初一,2013-12-03


In [10]:
PrintCCalender(1949)

一月初一,1949-01-29
二月初一,1949-02-28
三月初一,1949-03-29
四月初一,1949-04-28
五月初一,1949-05-28
六月初一,1949-06-26
七月初一,1949-07-26
潤七月初一,1949-08-24
八月初一,1949-09-22
九月初一,1949-10-22
十月初一,1949-11-20
十一月初一,1949-12-20


In [11]:
PrintCCalender(1948)

十二月初一,1948-01-11
一月初一,1948-02-10
二月初一,1948-03-11
三月初一,1948-04-09
四月初一,1948-05-09
五月初一,1948-06-07
六月初一,1948-07-07
七月初一,1948-08-05
八月初一,1948-09-03
九月初一,1948-10-03
十月初一,1948-11-01
十一月初一,1948-12-01
十二月初一,1948-12-30


In [12]:
# http://eclipse.gsfc.nasa.gov/eclipse.html 可查到歷年的日蝕月蝕的資料
def separation(a,b):
    val=a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))
    #if val>1.0:
    #    val=1.0
    #if val< -1.0:
    #    val= -1.0
    return(math.acos(val))
def ShadowCone(s,rs,o,ro,asin):
    o2s=s-o
    #dso=np.array([np.linalg.norm(x) for x in o2s])
    dso=np.linalg.norm(o2s,axis=1)
    sodir=np.array([-o2s[i]/dso[i] for i in range(len(o2s))])
    #sodir=np.diag(1.0/dso).dot(-o2s) ##### inefficient
    du=ro/(rs-ro)*dso
    ucenter=np.array([o[i]+du[i]*sodir[i] for i in range(len(o))])
    #ucenter=o+np.diag(du).dot(sodir) ##### inefficient
    #uarc=np.array([math.asin(rs/(dso[i]+du[i])) for i in range(len(s))])
    if asin:
        #uarc=np.array([math.asin(ro/du[i]) for i in range(len(s))])
        uarc=np.arcsin(ro/du)
    else:
        uarc=ro/du
    dp=ro/(rs+ro)*dso
    #parc=np.array([math.asin(rs/(dso[i]-dp[i])) for i in range(len(s))])
    if asin:
        #parc=np.array([math.asin(ro/dp[i]) for i in range(len(s))])
        parc=np.arcsin(ro/dp)
    else:
        parc=ro/dp
    pcenter=np.array([o[i]-dp[i]*sodir[i] for i in range(len(o))])
    #pcenter=o-np.diag(dp).dot(sodir) ##### inefficient
    return((ucenter,pcenter,uarc,parc,sodir))

def Region(S,O,T,rt,Ucenter,Pcenter,uarc,parc,SOdir):
    res=[]
    for i in range(len(T)):
        if (T[i]-O[i]).dot(SOdir[i])<=0:
            res.append('O')
            continue
        vp=T[i]-Pcenter[i]
        ptarc=math.asin(rt/np.linalg.norm(vp))
        psep=separation(T[i]-Pcenter[i],SOdir[i])
        if psep>parc[i]+ptarc:
            res.append('O')
            continue
        vu=T[i]-Ucenter[i]
        dist=np.linalg.norm(vu)
        if vu.dot(SOdir[i])<=0:
            if dist<=rt:
                res.append('F')
                continue
            utarc=math.asin(rt/dist)
            usep=separation(vu,-SOdir[i])
            if usep<= uarc[i]-utarc:
                res.append('G')
            elif usep<= uarc[i]+utarc:
                res.append('F')
            else:
                res.append('P')
            continue
        else:
            if dist<=rt:
                res.append('C')
                continue
            utarc=math.asin(rt/dist)
            usep=separation(vu,SOdir[i])
            if usep<= uarc[i]-utarc:
                res.append('D')
            elif usep<= uarc[i]+utarc:
                res.append('C')
            else:
                res.append('P')
            continue
    return(res)  

def EarthRegion(times,asin):
    gmttimes=times-eighthours
    sun=get_body('sun', gmttimes,  ephemeris='jpl')
    moon=get_body('moon', gmttimes,  ephemeris='jpl')
    earth=get_body('earth', gmttimes,  ephemeris='jpl')
    rs=const.R_sun.to('km').value
    re=const.R_earth.to('km').value
    ro=1737.1
    sunpos=np.transpose(np.array([sun.cartesian.x.value,
                                  sun.cartesian.y.value,sun.cartesian.z.value]))
    moonpos=np.transpose(np.array([moon.cartesian.x.value,
                                  moon.cartesian.y.value,moon.cartesian.z.value]))
    earthpos=np.zeros((len(sunpos),3))
    Ucenter,Pcenter,uarc,parc,SOdir=ShadowCone(sunpos,rs,moonpos,ro,asin)
    return(Region(sunpos,moonpos,earthpos,re,Ucenter,Pcenter,uarc,parc,SOdir))
    

def SolarEclipse(date,asin=False):
    times=Time(date)+oneminute*np.linspace(0,1440,1441)
    regions=EarthRegion(times,asin)
    print(times[0],regions[0])
    for i in range(len(times)-1):
        if regions[i]!=regions[i+1]:
            newsymbol=regions[i+1]
            sectimes=times[i]+onesecond*np.linspace(0,60,61)
            secregions=EarthRegion(sectimes,asin)
            for j in range(61):
                j1=60-j
                if secregions[j1]!=newsymbol:
                    print(sectimes[j1],newsymbol)
                    break
                    
    
SolarEclipse('2016-3-9',True)
SolarEclipse('2016-3-9',False)
SolarEclipse('2016-3-9')
SolarEclipse('2016-9-1')
SolarEclipse('2017-2-26')
SolarEclipse('2017-2-27')
SolarEclipse('2017-8-21')
SolarEclipse('2017-8-22')


2016-03-09 00:00:00.000 O
2016-03-09 07:19:20.000 P
2016-03-09 08:15:55.000 F
2016-03-09 11:38:26.000 P
2016-03-09 12:34:56.000 O
2016-03-09 00:00:00.000 O
2016-03-09 07:19:20.000 P
2016-03-09 08:15:55.000 F
2016-03-09 11:38:26.000 P
2016-03-09 12:34:56.000 O
2016-03-09 00:00:00.000 O
2016-03-09 07:19:20.000 P
2016-03-09 08:15:55.000 F
2016-03-09 11:38:26.000 P
2016-03-09 12:34:56.000 O
2016-09-01 00:00:00.000 O
2016-09-01 14:13:09.000 P
2016-09-01 15:17:50.000 C
2016-09-01 18:55:59.000 P
2016-09-01 20:00:42.000 O
2017-02-26 00:00:00.000 O
2017-02-26 20:10:42.000 P
2017-02-26 21:15:08.000 C
2017-02-27 00:00:00.000 C
2017-02-27 00:31:34.000 P
2017-02-27 01:35:57.000 O
2017-08-21 00:00:00.000 O
2017-08-21 23:46:49.000 P
2017-08-22 00:00:00.000 P
2017-08-22 00:48:28.000 F
2017-08-22 04:02:36.000 P
2017-08-22 05:04:21.000 O


In [13]:
def LunarRegion(times,asin):
    gmttimes=times-eighthours
    sun=get_body('sun', gmttimes,  ephemeris='jpl')
    moon=get_body('moon', gmttimes,  ephemeris='jpl')
    earth=get_body('earth', gmttimes,  ephemeris='jpl')
    rs=const.R_sun.to('km').value
    re=const.R_earth.to('km').value
    rm=1737.1
    sunpos=np.transpose(np.array([sun.cartesian.x.value,
                                  sun.cartesian.y.value,sun.cartesian.z.value]))
    moonpos=np.transpose(np.array([moon.cartesian.x.value,
                                  moon.cartesian.y.value,moon.cartesian.z.value]))
    earthpos=np.zeros((len(sunpos),3))
    # reference http://eclipse.gsfc.nasa.gov/LEcat5/shadow.html
    # NASA 用 Danjon 的方法，將地球放大成 1.01倍，中央氣象局用 Chauvenet 的方法，類似於將地球放大1.02倍
    # 此處採用Danjon 的方法
    Ucenter,Pcenter,uarc,parc,SOdir=ShadowCone(sunpos,rs,earthpos,1.01*re,asin)
    return(Region(sunpos,earthpos,moonpos,rm,Ucenter,Pcenter,uarc,parc,SOdir))

def LunarEclipse(date,asin=False):
    times=Time(date)+oneminute*np.linspace(0,1440,1441)
    regions=LunarRegion(times,asin)
    print(times[0],regions[0])
    for i in range(len(times)-1):
        if regions[i]!=regions[i+1]:
            newsymbol=regions[i+1]
            sectimes=times[i]+onesecond*np.linspace(0,60,61)
            secregions=LunarRegion(sectimes,asin)
            for j in range(61):
                j1=60-j
                if secregions[j1]!=newsymbol:
                    print(sectimes[j1],newsymbol)
                    break
LunarEclipse('2016-3-23',True)
LunarEclipse('2016-3-23',False)
LunarEclipse('2016-3-23')
LunarEclipse('2016-9-17')
LunarEclipse('2016-9-16')
LunarEclipse('2017-2-11')
LunarEclipse('2017-8-7')
LunarEclipse('2017-8-8')


2016-03-23 00:00:00.000 O
2016-03-23 17:39:30.000 P
2016-03-23 21:54:54.000 O
2016-03-23 00:00:00.000 O
2016-03-23 17:39:30.000 P
2016-03-23 21:54:54.000 O
2016-03-23 00:00:00.000 O
2016-03-23 17:39:30.000 P
2016-03-23 21:54:54.000 O
2016-09-17 00:00:00.000 O
2016-09-17 00:54:43.000 P
2016-09-17 04:54:02.000 O
2016-09-16 00:00:00.000 O
2017-02-11 00:00:00.000 O
2017-02-11 06:34:17.000 P
2017-02-11 10:53:21.000 O
2017-08-07 00:00:00.000 O
2017-08-07 23:50:02.000 P
2017-08-08 00:00:00.000 P
2017-08-08 01:22:55.000 F
2017-08-08 03:18:07.000 P
2017-08-08 04:50:53.000 O


In [14]:
import inspect
#inspect.getargspec(SkyCoord.transform_to)
print(inspect.signature(SkyCoord.transform_to))
print(get_body('sun',Time('2016-8-25')).distance.to('m')/const.c)
print(get_body('moon',Time('2016-8-25')).distance.to('m')/const.c)


(self, frame, merge_attributes=True)
504.3839492816898 s
1.2357848055638174 s


In [15]:
M=np.random.rand(1440,3)
d=np.random.rand(1440)


In [16]:
timeit np.diag(d).dot(M)

5.08 ms ± 317 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
timeit np.array([d[i]*M[i] for i in range(len(d))])


4.23 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%magic

In [19]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python2  %%py

In [20]:
np.__version__
np.array??